In [1]:
def get_densities(mode):
    if mode == False:
        pass
    elif mode == True:
        # initialize correlation lists
        corr_1870 = []
        corr_1979 = []

        # initialize model PDF lists
        model_pdf_1870_2014 = []
        model_pdf_1979_2014 = []
        model_pdf_1870_2023 = []
        model_pdf_1979_2023 = []

        # initialize obs PDF lists
        obs_pdf_2014 = []
        obs_pdf_2023 = []

        for i in range(0,len(data_var)):
            ## Get correlation and regression parameters (to be used for the KDE)
            corr_1870_calc = getCorr(global_mean(toa_e[data_var[i]]).max(dim='lag'),
                                     global_mean(toa_a[data_var[i]]-toa_h_1870[data_var[i]]))
            corr_1979_calc = getCorr(global_mean(toa_e[data_var[i]]).max(dim='lag'),
                                     global_mean(toa_a[data_var[i]]-toa_h_1979[data_var[i]]))
            params_1870 = getReg(global_mean(toa_e[data_var[i]]).max(dim='lag'),
                                 global_mean(toa_a[data_var[i]]-toa_h_1870[data_var[i]]))
            params_1979 = getReg(global_mean(toa_e[data_var[i]]).max(dim='lag'),
                                 global_mean(toa_a[data_var[i]]-toa_h_1979[data_var[i]]))

            ## Append correlations to list
            corr_1870.append(corr_1870_calc)
            corr_1979.append(corr_1979_calc)

            ## Calculate model KDEs
            model_pdf_1870_2014.append(get_kde(n=int(10e5),
                                               x=toa_eo_2014_gm[data_var[i]].max(dim='lag').values,
                                               params=params_1870,y=np.linspace(-1,3,100)))
            model_pdf_1979_2014.append(get_kde(n=int(10e5),
                                               x=toa_eo_2014_gm[data_var[i]].max(dim='lag').values,
                                               params=params_1979,y=np.linspace(-1,3,100)))
            model_pdf_1870_2023.append(get_kde(n=int(10e5),
                                               x=toa_eo_2023_gm[data_var[i]].max(dim='lag').values,
                                               params=params_1870,y=np.linspace(-1,3,100)))
            model_pdf_1979_2023.append(get_kde(n=int(10e5),
                                               x=toa_eo_2023_gm[data_var[i]].max(dim='lag').values,
                                               params=params_1979,y=np.linspace(-1,3,100)))

             ## Calculate obs PDFs
            loc_2014 = toa_eo_2014_gm[data_var[i]].max(dim='lag')
            loc_2023 = toa_eo_2023_gm[data_var[i]].max(dim='lag')
            scale_2014 = toa_eo_68ci_2014_max_gm.sel(ci='upper')[data_var_ci[i]] - toa_eo_2014_gm[data_var[i]].max(dim='lag').values
            scale_2023 = toa_eo_68ci_2023_max_gm.sel(ci='upper')[data_var_ci[i]] - toa_eo_2023_gm[data_var[i]].max(dim='lag').values
            obs_pdf_2014.append(get_pdf(x=np.linspace(-0.1,0.5,100),
                                        loc=loc_2014,
                                        scale=scale_2014))
            obs_pdf_2023.append(get_pdf(x=np.linspace(-0.1,0.5,100),
                                        loc=loc_2023,
                                        scale=scale_2023))
            
        return corr_1870, corr_1979, params_1870, params_1979, model_pdf_1870_2014, model_pdf_1979_2014, model_pdf_1870_2023, model_pdf_1979_2023, obs_pdf_2014, obs_pdf_2023

In [ ]:
if emer_const_mode == True:
    corr_1870, corr_1979, params_1870, params_1979, model_pdf_1870_2014, model_pdf_1979_2014, model_pdf_1870_2023, model_pdf_1979_2023, obs_pdf_2014, obs_pdf_2023 = get_densities(emer_const_mode)

In [ ]:
fig, ax = plt.subplots(3,5,figsize=(24,12))
fig.tight_layout(h_pad=5)
k = 0
lag_list = np.arange(-12,13)
title = ['All Sky','Clear Sky','Net CRE','SW CRE','LW CRE']
data_var_ci = ['all_sky_ci','clr_sky_ci','net_cre_ci','sw_cre_ci','lw_cre_ci']

def reg_plot(x,y,j,k):
    # plot the regression line fit using seaborn
    sns.regplot(x=x, y=y, ax=ax[j,k], scatter_kws={"color": "blue"}, line_kws={"color": "blue"}, truncate=False)

def pdf_plot(model_pdf,obs_pdf_2014,obs_pdf_2023,j,k):
    # plot the model PDFs (scaled arbitrarily)
    ax[j,k].plot(model_pdf[k]/np.max(model_pdf[k])*0.12+ax[j,k].get_xlim()[0],np.linspace(-1,3,100),color='k')
    ax[j,k].plot(model_pdf[k]/np.max(model_pdf[k])*0.12+ax[j,k].get_xlim()[0],np.linspace(-1,3,100),color='k',linestyle='dashed')
    # plot the observational PDFs (scaled arbitrarily)
    if k == 4:
        ax[j,k].plot(np.linspace(-0.1,0.5,100),obs_pdf_2014[k]/np.mean(obs_pdf_2014[k])*0.09+ax[j,k].get_ylim()[0],color='r')
        ax[j,k].plot(np.linspace(-0.1,0.5,100),obs_pdf_2023[k]/np.mean(obs_pdf_2023[k])*0.09+ax[j,k].get_ylim()[0],color='r',
                     linestyle='dashed')
    else:
        ax[j,k].plot(np.linspace(-0.1,0.5,100),obs_pdf_2014[k]/np.mean(obs_pdf_2014[k])*0.12+ax[j,k].get_ylim()[0],color='r')
        ax[j,k].plot(np.linspace(-0.1,0.5,100),obs_pdf_2023[k]/np.mean(obs_pdf_2023[k])*0.12+ax[j,k].get_ylim()[0],color='r',
                     linestyle='dashed')

def emerconst_plot(x,x_ci,p,j,k):
    # plot the expected value and 95% conf int for obs ENSO feedbacks and pattern effect using emergent constraint
    ax[j,k].vlines(x=x.max(dim='lag'),ymax=y_max(x.max(dim='lag'),p),
                   ymin=-1,color='r')
    ax[j,k].vlines(x=x_ci[0],ymin=-1,ymax=y_max(x_ci[0],p),
                   color='r')
    ax[j,k].vlines(x=x_ci[1],ymin=-1,ymax=y_max(x_ci[1],p),
                   color='r')
    ax[j,k].hlines(y=y_max(x.max(dim='lag'),p),xmax=x.max(dim='lag'),
                   xmin=-0.1,color='k')
    ax[j,k].hlines(y=y_max(x_ci[0],p),xmin=-0.1,xmax=x_ci[0],
                   color='k')
    ax[j,k].hlines(y=y_max(x_ci[1],p),xmin=-0.1,xmax=x_ci[1],
                   color='k')

for i in data_var:
    ## Calculate regression line for each feedback
    c = sm.add_constant(global_mean(toa_e[i]).max(dim='lag').values)
    p1 = sm.OLS(global_mean(toa_a[i]-toa_h_1870[i]).values,c).fit().params
    p2 = sm.OLS(global_mean(toa_a[i]-toa_h_1979[i]).values,c).fit().params
    def y_max(x,p):
        return x*p[1] + p[0]
    
    # Set the x/y limits to force seaborn to extrapolate the regression line to these limits
    for j in range(0,2):
        ax[j,k].set_xlim([-0.1,0.5])
        ax[j,k].set_ylim([-1,3])
        ax[j,2].set_xlabel('$\lambda_{ENSO}$ [$W/m^2/K$]',size=15)
    
    # [0,k] = 1870-2014 emer const plot
    # plot the regression line fit using seaborn
    reg_plot(x=global_mean(toa_e[i]).max(dim='lag').values,
             y=global_mean(toa_a[i]-toa_h_1870[i]).values,
             j=0,k=k)

    # add the probability densities
    pdf_plot(model_pdf=model_pdf_1870_2014,
             obs_pdf_2014=obs_pdf_2014,
             obs_pdf_2023=obs_pdf_2023,
             j=0,k=k)

    # add the emergent constraint lines
    emerconst_plot(x=toa_eo_2014_gm[i],
                   x_ci=toa_eo_ci_2014_max_gm[data_var_ci[k]],
                   p=p1,
                   j=0,k=k)

    ax[0,k].set_title(title[k]+' (r = '+str(np.round(corr_1870[k],2))+')',size=15)
    ax[0,0].set_ylabel('$\Delta\lambda$ (1870 - 2014) [$W/m^2/K$]',size=15);

    # [1,k] = 1979-2014 emer const plot
    # plot the regression line fit using seaborn
    reg_plot(x=global_mean(toa_e[i]).max(dim='lag').values,
             y=global_mean(toa_a[i]-toa_h_1979[i]).values,
             j=1,k=k)

    # add the probability densities
    pdf_plot(model_pdf=model_pdf_1979_2014,
             obs_pdf_2014=obs_pdf_2014,
             obs_pdf_2023=obs_pdf_2023,
             j=1,k=k)

    # add the emergent constraint lines
    emerconst_plot(x=toa_eo_2014_gm[i],
                   x_ci=toa_eo_ci_2014_max_gm[data_var_ci[k]],
                   p=p2,
                   j=1,k=k)

    ax[1,k].set_title(title[k]+' (r = '+str(np.round(corr_1979[k],2))+')',size=15)
    ax[1,0].set_ylabel('$\Delta\lambda$ (1979 - 2014) [$W/m^2/K$]',size=15);

    # [2,k] = lagged ENSO feedbacks plot
    ax[2,k].plot(lag_list,global_mean(toa_e[i]).T,color='gray',alpha=0.5)
    ax[2,k].plot(lag_list,global_mean(toa_e[i]).mean(dim='model'),color='k')
    ax[2,k].plot(lag_list,(toa_eo_2014_gm[i]),color='r')
    ax[2,k].fill_between(lag_list,toa_eo_ci_2014_gm[data_var_ci[k]].sel(ci='lower'),toa_eo_ci_2014_gm[data_var_ci[k]].sel(ci='upper'),
                        color='r',alpha=0.2)
    ax[2,k].axhline(y=0,linestyle='dotted',color='k',alpha=0.5)
    ax[2,k].axvline(x=0,linestyle='dotted',color='k',alpha=0.5)

    ax[2,k].set_title(title[k],size=15)
    ax[2,k].set_ylim([-0.6,0.6])
    ax[2,k].set_xlim([lag_list[0],lag_list[-1]])
    ax[2,2].set_xlabel('Lag [Months]',size=15)
    ax[2,0].set_ylabel('$\lambda_{ENSO}$ (2001-2014) [$W/m^2/K$]',size=15)

    k = k+1
    
path = '/data/keeling/a/tjhanke2/enso-as-an-emergent-constraint/outputs/'
fig.savefig(path+'emergent_constraint_delta_lambda_lambda_enso.png',bbox_inches='tight')